In [ ]:
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

#데이터셋 임포트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/[DScover] /25-2 Guide Project/train_preprocessed (2).csv')
df_test = pd.read_csv('/content/drive/MyDrive/[DScover] /25-2 Guide Project/test_preprocessed.csv')

In [ ]:
df_train.columns

Index(['Unnamed: 0', 'building_id', 'date', 'precipitation', 'windspeed',
       'power_consumption', 'building_type', 'total_floor_area',
       'solar_capacity', 'hour', 'dow', 'month', 'day', 'is_weekend',
       'cooling_ratio', 'THI', 'has_pv', 'has_ess', 'CDH', 'sin_hour',
       'cos_hour', 'sin_date', 'cos_date', 'sin_month', 'cos_month',
       'sin_dayofweek', 'cos_dayofweek', 'day_hour_mean', 'day_hour_std',
       'hour_mean', 'hour_std', 'temp_above_24', 'temp_above_28',
       'solar_exposure'],
      dtype='object')

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203932 entries, 0 to 203931
Data columns (total 34 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         203932 non-null  int64  
 1   building_id        203932 non-null  int64  
 2   date               203932 non-null  object 
 3   precipitation      203932 non-null  float64
 4   windspeed          203932 non-null  float64
 5   power_consumption  203932 non-null  float64
 6   building_type      203932 non-null  object 
 7   total_floor_area   203932 non-null  float64
 8   solar_capacity     203932 non-null  float64
 9   hour               203932 non-null  int64  
 10  dow                203932 non-null  int64  
 11  month              203932 non-null  int64  
 12  day                203932 non-null  int64  
 13  is_weekend         203932 non-null  int64  
 14  cooling_ratio      203932 non-null  float64
 15  THI                203932 non-null  float64
 16  ha

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            16800 non-null  int64  
 1   building_id           16800 non-null  int64  
 2   date                  16800 non-null  object 
 3   precipitation         16800 non-null  float64
 4   windspeed             16800 non-null  float64
 5   building_type         16800 non-null  object 
 6   total_floor_area      16800 non-null  float64
 7   solar_capacity        16800 non-null  float64
 8   hour                  16800 non-null  int64  
 9   dow                   16800 non-null  int64  
 10  month                 16800 non-null  int64  
 11  day                   16800 non-null  int64  
 12  is_weekend            16800 non-null  int64  
 13  cooling_ratio         16800 non-null  float64
 14  THI                   16800 non-null  float64
 15  has_pv             

In [ ]:
df_train['building_type']

,building_type
0,호텔
1,호텔
2,호텔
3,호텔
4,호텔
...,...
203927,호텔
203928,호텔
203929,호텔
203930,호텔


##데이터셋 추가 전처리

In [ ]:
# 불필요한 컬럼 드랍
drop_cols_train = ["Unnamed: 0", "date"]
drop_cols_test = ["Unnamed: 0", "date", "pred_sunshine_hours", "pred_solar_radiation"]

# === 학습/테스트에서 공통으로 제거 ===
df_train_clean = df_train.drop(columns=drop_cols_train, errors="ignore")
df_test_clean = df_test.drop(columns=drop_cols_test, errors="ignore")

print("Before:", df_train.shape, df_test.shape)
print("After :", df_train_clean.shape, df_test_clean.shape)

Before: (203932, 34) (16800, 35)
After : (203932, 32) (16800, 31)


In [ ]:
target_col = "power_consumption"
X = df_train_clean.drop(columns=[target_col])
y = df_train_clean[target_col].values.reshape(-1, 1)


In [ ]:
X_test = df_test_clean.copy()

In [ ]:
import numpy as np

def add_time_features(df):
    # hour (0~23)
    if "hour" in df.columns:
        df["sin_hour"] = np.sin(2 * np.pi * df["hour"] / 24)
        df["cos_hour"] = np.cos(2 * np.pi * df["hour"] / 24)

    # day (1~31)
    if "day" in df.columns:
        df["sin_day"] = np.sin(2 * np.pi * df["day"] / 31)
        df["cos_day"] = np.cos(2 * np.pi * df["day"] / 31)

    # month (1~12)
    if "month" in df.columns:
        df["sin_month"] = np.sin(2 * np.pi * df["month"] / 12)
        df["cos_month"] = np.cos(2 * np.pi * df["month"] / 12)

    # day of week (0~6)
    if "dow" in df.columns:
        df["sin_dow"] = np.sin(2 * np.pi * df["dow"] / 7)
        df["cos_dow"] = np.cos(2 * np.pi * df["dow"] / 7)

    return df

# === Train / Test 모두 적용 ===
X = add_time_features(X)
X_test = add_time_features(X_test)

In [ ]:
# 타겟 분리
target_col = "power_consumption"
y = df_train[target_col]

In [ ]:
# 컬럼 구분
categorical_features = ["building_type"]  # 원핫 인코딩 대상
numeric_features = [col for col in X.columns if col not in categorical_features]

In [ ]:
# 스케일링 대상 숫자형 피처 선정
# (0/1 변수는 스케일링 불필요 → 제외)
binary_like = ["is_weekend", "has_pv", "has_ess"]
numeric_to_scale = [col for col in numeric_features if col not in binary_like]

In [ ]:
#스케일러 정의
numeric_transformer = RobustScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
y_scaler = StandardScaler()

In [ ]:
# ColumnTransformer 구성
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_to_scale),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="passthrough"  # 스케일링 안 하는 피처는 그대로 둠
)

# 파이프라인
X_pipeline = Pipeline(steps=[("preprocessor", preprocessor)])


In [ ]:
y = df_train[target_col].to_numpy().reshape(-1, 1)

In [ ]:
X_scaled = X_pipeline.fit_transform(X)
X_test_scaled = X_pipeline.transform(X_test)
y_scaled = y_scaler.fit_transform(y)

In [ ]:
y_scaled

array([[ 0.66792498],
       [ 0.61291316],
       [ 0.54415043],
       ...,
       [-0.08784707],
       [-0.18458329],
       [-0.10879466]])

#1D CNN, ResNet 모델링

In [ ]:
# 시드 값
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# PyTorch 재현성 모드
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
##데이터 변환
X_tensor = torch.tensor(X_scaled_pca, dtype=torch.float32).unsqueeze(1)  # (n, 1, n_components)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled_pca, dtype=torch.float32).unsqueeze(1)

print("Train X shape:", X_tensor.shape)  # (n, 1, n_components)
print("Train y shape:", y_tensor.shape)  # (n, 1)
print("Test X shape:", X_test_tensor.shape)

# Dataset/DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256)

Train X shape: torch.Size([203932, 1, 15])
Train y shape: torch.Size([203932, 1])
Test X shape: torch.Size([16800, 1, 15])


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(ResidualBlock, self).__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=1, padding=padding)
        self.bn2 = nn.BatchNorm1d(out_channels)

        # skip connection (채널/stride 다르면 projection 사용)
        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return torch.relu(out)

In [ ]:
class ResNetRegressor(nn.Module):
    def __init__(self, input_len):
        super(ResNetRegressor, self).__init__()
        self.layer1 = ResidualBlock(1, 32)
        self.pool1 = nn.MaxPool1d(2)

        self.layer2 = ResidualBlock(32, 64)
        self.pool2 = nn.MaxPool1d(2)

        conv_out_len = input_len // 4  # 두 번 pooling

        self.fc1 = nn.Linear(64 * conv_out_len, 128)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool1(self.layer1(x))
        x = self.pool2(self.layer2(x))
        x = x.view(x.size(0), -1)   # Flatten
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResNetRegressor(input_len=X_scaled_pca.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 50
for epoch in range(epochs):
    # --- Train ---
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * X_batch.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 1/50, Train Loss: 0.0492, Val Loss: 0.0183
Epoch 2/50, Train Loss: 0.0273, Val Loss: 0.0131
Epoch 3/50, Train Loss: 0.0242, Val Loss: 0.0129
Epoch 4/50, Train Loss: 0.0234, Val Loss: 0.0140
Epoch 5/50, Train Loss: 0.0220, Val Loss: 0.0109
Epoch 6/50, Train Loss: 0.0214, Val Loss: 0.0099
Epoch 7/50, Train Loss: 0.0207, Val Loss: 0.0181
Epoch 8/50, Train Loss: 0.0203, Val Loss: 0.0109
Epoch 9/50, Train Loss: 0.0202, Val Loss: 0.0130
Epoch 10/50, Train Loss: 0.0196, Val Loss: 0.0098
Epoch 11/50, Train Loss: 0.0189, Val Loss: 0.0091
Epoch 12/50, Train Loss: 0.0189, Val Loss: 0.0107
Epoch 13/50, Train Loss: 0.0187, Val Loss: 0.0094
Epoch 14/50, Train Loss: 0.0181, Val Loss: 0.0085
Epoch 15/50, Train Loss: 0.0181, Val Loss: 0.0084
Epoch 16/50, Train Loss: 0.0181, Val Loss: 0.0115
Epoch 17/50, Train Loss: 0.0178, Val Loss: 0.0086
Epoch 18/50, Train Loss: 0.0174, Val Loss: 0.0100
Epoch 19/50, Train Loss: 0.0172, Val Loss: 0.0114
Epoch 20/50, Train Loss: 0.0174, Val Loss: 0.0122
Epoch 21/

In [ ]:
model.eval()
with torch.no_grad():
    y_test_pred_scaled = model(X_test_tensor.to(device)).cpu().numpy()

# y 복원
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled)

print("테스트 세트 예측 샘플:", y_test_pred[:5].flatten())

테스트 세트 예측 샘플: [3666.0244 3448.2712 3429.3237 3421.4944 3339.2703]


In [ ]:
sub4 = pd.DataFrame(y_test_pred)

In [ ]:
sub4.to_csv('submission_04_jwp.csv')

#Simple LSTM

In [ ]:
# numpy → tensor 변환
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

# LSTM 입력 모양: (batch, seq_len, features)
X_tensor = X_tensor.unsqueeze(1)  # (n, 1, feature_dim)

print("X_tensor:", X_tensor.shape)  # (샘플 수, 1, 피처 수)
print("y_tensor:", y_tensor.shape)  # (샘플 수, 1)

# Dataset & DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256)

X_tensor: torch.Size([203932, 1, 40])
y_tensor: torch.Size([203932, 1])


In [ ]:
class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super(LSTMRegressor, self).__init__()
        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # 회귀 출력

    def forward(self, x):
        # x: (batch, seq_len=1, features)
        out, _ = self.lstm(x)  # out: (batch, seq_len, hidden_dim)
        out = out[:, -1, :]    # 마지막 시점 hidden state
        out = self.fc(out)
        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMRegressor(input_dim=X_scaled.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 20
for epoch in range(epochs):
    # --- Train ---
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * X_batch.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 1/20, Train Loss: 0.0716, Val Loss: 0.0114
Epoch 2/20, Train Loss: 0.0107, Val Loss: 0.0096
Epoch 3/20, Train Loss: 0.0094, Val Loss: 0.0086
Epoch 4/20, Train Loss: 0.0085, Val Loss: 0.0081
Epoch 5/20, Train Loss: 0.0077, Val Loss: 0.0073
Epoch 6/20, Train Loss: 0.0072, Val Loss: 0.0072
Epoch 7/20, Train Loss: 0.0070, Val Loss: 0.0074
Epoch 8/20, Train Loss: 0.0066, Val Loss: 0.0070
Epoch 9/20, Train Loss: 0.0065, Val Loss: 0.0064
Epoch 10/20, Train Loss: 0.0063, Val Loss: 0.0070
Epoch 11/20, Train Loss: 0.0061, Val Loss: 0.0062
Epoch 12/20, Train Loss: 0.0059, Val Loss: 0.0056
Epoch 13/20, Train Loss: 0.0058, Val Loss: 0.0055
Epoch 14/20, Train Loss: 0.0057, Val Loss: 0.0057
Epoch 15/20, Train Loss: 0.0056, Val Loss: 0.0053
Epoch 16/20, Train Loss: 0.0054, Val Loss: 0.0056
Epoch 17/20, Train Loss: 0.0053, Val Loss: 0.0053
Epoch 18/20, Train Loss: 0.0052, Val Loss: 0.0057
Epoch 19/20, Train Loss: 0.0051, Val Loss: 0.0050
Epoch 20/20, Train Loss: 0.0050, Val Loss: 0.0053


In [ ]:
# 검증 세트 예측
model.eval()
with torch.no_grad():
    X_val, y_val = next(iter(val_loader))
    X_val, y_val = X_val.to(device), y_val.to(device)
    y_pred_scaled = model(X_val).cpu().numpy()

# y 복원
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_true = y_scaler.inverse_transform(y_val.cpu().numpy())

print("예측 복원 샘플:", y_pred[:5].flatten())
print("실제값 샘플:", y_true[:5].flatten())

예측 복원 샘플: [ 565.231  3116.078   529.8936 1586.8206 1595.9186]
실제값 샘플: [ 542.16   3223.08    608.16   1687.6799 1556.6399]


In [ ]:
sub2 = pd.DataFrame(y_pred)

In [ ]:
sub2.to_csv('submission_02_jwp.csv')

#TCN

In [ ]:
import numpy as np

def create_sliding_window(X, y, window_size=24):
    """
    X: (n_samples, n_features)
    y: (n_samples, 1)
    window_size: 몇 시점씩 묶을지
    """
    Xs, ys = [], []
    for i in range(len(X) - window_size):
        Xs.append(X[i:i+window_size])   # 입력 시퀀스
        ys.append(y[i+window_size])     # 다음 시점 target
    return np.array(Xs), np.array(ys)

In [ ]:
window_size = 24  # 하루 단위
X_seq, y_seq = create_sliding_window(X_scaled, y_scaled, window_size)

# Torch tensor 변환 (TCN은 (batch, features, seq_len) 필요)
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

X_seq_torch = torch.tensor(X_seq, dtype=torch.float32).permute(0, 2, 1)  # (batch, features, seq_len)
y_seq_torch = torch.tensor(y_seq, dtype=torch.float32)

print("Train X shape:", X_seq_torch.shape)
print("Train y shape:", y_seq_torch.shape)

# Dataset/DataLoader
dataset = TensorDataset(X_seq_torch, y_seq_torch)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256)

Train X shape: torch.Size([203908, 44, 24])
Train y shape: torch.Size([203908, 1])


In [ ]:
import torch.nn as nn

class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1, dropout=0.2):
        super(TemporalBlock, self).__init__()
        padding = (kernel_size - 1) * dilation // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=padding, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=padding, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        # skip connection
        self.downsample = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None

    def forward(self, x):
        out = self.dropout1(self.relu1(self.bn1(self.conv1(x))))
        out = self.dropout2(self.relu2(self.bn2(self.conv2(out))))
        res = x if self.downsample is None else self.downsample(x)
        return torch.relu(out + res)

class TCNRegressor(nn.Module):
    def __init__(self, input_channels, num_channels=[64, 128, 128], kernel_size=3, dropout=0.2):
        super(TCNRegressor, self).__init__()
        layers = []
        for i in range(len(num_channels)):
            in_ch = input_channels if i == 0 else num_channels[i-1]
            out_ch = num_channels[i]
            layers.append(TemporalBlock(in_ch, out_ch, kernel_size, dilation=2**i, dropout=dropout))
        self.tcn = nn.Sequential(*layers)
        self.fc = nn.Linear(num_channels[-1], 1)

    def forward(self, x):
        out = self.tcn(x)          # (batch, channels, seq_len)
        out = out.mean(dim=2)      # Global average pooling
        out = self.fc(out)
        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TCNRegressor(input_channels=X_seq_torch.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 20
for epoch in range(epochs):
    # --- Train ---
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * X_batch.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 1/20, Train Loss: 0.0493, Val Loss: 0.0201
Epoch 2/20, Train Loss: 0.0219, Val Loss: 0.0143
Epoch 3/20, Train Loss: 0.0168, Val Loss: 0.0108
Epoch 4/20, Train Loss: 0.0142, Val Loss: 0.0104
Epoch 5/20, Train Loss: 0.0132, Val Loss: 0.0100
Epoch 6/20, Train Loss: 0.0121, Val Loss: 0.0092
Epoch 7/20, Train Loss: 0.0113, Val Loss: 0.0093
Epoch 8/20, Train Loss: 0.0104, Val Loss: 0.0084
Epoch 9/20, Train Loss: 0.0096, Val Loss: 0.0084
Epoch 10/20, Train Loss: 0.0092, Val Loss: 0.0099
Epoch 11/20, Train Loss: 0.0090, Val Loss: 0.0066
Epoch 12/20, Train Loss: 0.0084, Val Loss: 0.0077
Epoch 13/20, Train Loss: 0.0082, Val Loss: 0.0065
Epoch 14/20, Train Loss: 0.0078, Val Loss: 0.0060
Epoch 15/20, Train Loss: 0.0072, Val Loss: 0.0066
Epoch 16/20, Train Loss: 0.0071, Val Loss: 0.0061
Epoch 17/20, Train Loss: 0.0068, Val Loss: 0.0066
Epoch 18/20, Train Loss: 0.0064, Val Loss: 0.0056
Epoch 19/20, Train Loss: 0.0064, Val Loss: 0.0059
Epoch 20/20, Train Loss: 0.0061, Val Loss: 0.0062


In [ ]:
# test 데이터에도 동일하게 sliding window 적용
X_test_seq = []
for i in range(len(X_test_scaled) - window_size):
    X_test_seq.append(X_test_scaled[i:i+window_size])
X_test_seq = np.array(X_test_seq)

X_test_seq_torch = torch.tensor(X_test_seq, dtype=torch.float32).permute(0, 2, 1).to(device)

# 예측
model.eval()
with torch.no_grad():
    y_test_pred_scaled = model(X_test_seq_torch).cpu().numpy()

# 스케일 복원
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled)

print("테스트 세트 예측 샘플:", y_test_pred[:5].flatten())

테스트 세트 예측 샘플: [4316.558  4127.894  3973.6975 3808.317  3701.1484]


In [ ]:
sub6 = pd.DataFrame(y_test_pred)

In [ ]:
sub6.to_csv('submission_06_jwp.csv')